Import et préparation des données :

In [1]:
# import des librairies
import nltk
import numpy as np
import random
import string # to process standard python strings
import re
import spacy

In [2]:


# import du texte & nettoyages
f=open('infos_corona.txt','r',errors = 'ignore', encoding = "utf8")
raw=f.read()

raw = re.sub('n.c.a.', 'nca', raw)

# Création d'une liste de phrases (= tokenization)
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw, language="french")

In [3]:
sent_tokens

['\ufeffDepuis janvier 2020, une épidémie de Coronavirus COVID-19 (ex 2019-nCoV) s’est propagée depuis la Chine.',
 'Retrouvez sur cette plateforme toutes les réponses officielles aux questions que vous vous posez sur ce qu’est le Coronavirus COVID-19 et les recommandations pour votre santé et votre vie quotidienne.',
 'Face à l’épidémie , il existe des gestes simples pour préserver votre santé et celle de votre entourage :\n\nSe laver les mains très régulièrement\nTousser ou éternuer dans son coude ou dans un mouchoir \nSaluer sans se serrer la main, arrêter les embrassades\nUtiliser des mouchoirs à usage unique et les jeter\nEviter les rassemblements, limiter les déplacements et les contacts.',
 'Qu’est-ce que le Coronavirus COVID-19 ?',
 'Les coronavirus sont une grande famille de virus, qui provoquent des maladies allant d’un simple rhume (certains virus saisonniers sont des Coronavirus) à des pathologies plus sévères comme le MERS-COV ou le SRAS.',
 'Le virus identifié en janvier 

In [4]:
for i in sorted(range(len(sent_tokens)), reverse = True):
    if re.search(r"\?", sent_tokens[i]):
        del sent_tokens[i]

In [5]:
# on enlève les doublons
sent_tokens = list(set(sent_tokens)) 

In [6]:
# On crée une liste nettoyée mais qui ne sera pas celle dans laquelle
# on ira chercher les réponses, simplement pour la création de la
# matrice TF-IDF
def nettoyage(texte):
    #texte = texte.lower() #pas necessaire
    # on remplace covid-19 par coronavirus
    
    
    texte = re.sub('covid-19| virus|covid 19 |sars-cov', 'coronavirus', texte)
    # on remplace les "coronavirus coronavirus" par coronavirus
    texte = re.sub('coronavirus coronavirus', 'coronavirus', texte)
    texte = re.sub(f"[{string.punctuation}]", " ", texte)
    texte = re.sub('[éèê]', 'e', texte)
    texte = re.sub('[àâ]', 'a', texte)
    texte = re.sub('[ô]', 'o', texte)
    texte = re.sub('mort(\w){0,3}|deces|deced(\w){1,5}', 'deces', texte)
    texte = re.sub('medec(\w){1,5}|medic{1,5}', 'medical', texte)
    return texte

In [7]:
phrases_nettoyees = []
for i in range(len(sent_tokens)):
    phrases_nettoyees.append(nettoyage(sent_tokens[i]))


nlp = spacy.load('fr_core_news_md')


doc = nlp(u"Il n’existe pas de vaccin contre le Coronavirus COVID-19 pour le moment.',")
for token in doc:
    print(token, token.lemma_)

#import spacy

class LemmaTokenizer(object):
    def __init__(self):
        self.spacynlp = spacy.load('fr_core_news_md')
    def __call__(self, doc):
        nlpdoc = self.spacynlp(doc)
        nlpdoc = [token.lemma_ for token in nlpdoc if (len(token.lemma_) > 1) or (token.lemma_.isalnum()) ]
        return nlpdoc

#vect = TfidfVectorizer(tokenizer=LemmaTokenizer())
#vect.fit(['On voit dans ce notebook comment créer '])
#print(vect.vocabulary_) # show the word-matrix position pairs
### prints {'on': 5, 'voir': 6, 'dans': 3, 'ce': 0, 'notebook': 4, 'comment': 1, 'créer': 2}

Calcul matrice TF-IDF :

In [8]:
lemmer = nltk.stem.snowball.FrenchStemmer()
def LemTokens(tokens):
    return [lemmer.stem(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
#un exemple :
#LemNormalize("On définit maintenant une fonction à indiquer dans le paramètre tokenizer de la fonction TfidfVectorizer. Cette fonction doit récupérer la racine des mots plutôt que les mots en entier pour trouver des correspondances entre des mots de la même racine même s'ils ne sont pas écrits sous la même forme")

In [9]:

LemNormalize("se laver les main, me lave les mains")

['se', 'lav', 'le', 'main', 'me', 'lav', 'le', 'main']

In [10]:
# Entraînement d'une matrice TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words
french_stop_words = get_stop_words('french')
french_stop_words.append('lundi')


TfidfVec = TfidfVectorizer( stop_words = french_stop_words)
tfidf = TfidfVec.fit(phrases_nettoyees)
# on crée la matrice TF-IDF sur le texte de la page wiki
phrases_tf = tfidf.transform(phrases_nettoyees)
#print(list(tfidf.vocabulary_))

In [11]:
print(TfidfVec.get_feature_names())

['000', '018', '08h00', '10', '100', '11', '110', '12', '120', '12h00', '13', '130', '135', '14', '15', '150', '16', '17', '18', '19', '1er', '1g', '20', '2009', '2019', '2020', '21', '22', '220', '23', '24', '24h', '25', '30', '31', '36', '38', '3eme', '3g', '48h', '50', '500', '53', '56', '60', '65', '674', '689', '6h', '723', '738', '746', '75', '80', '800', '824', '907', '91', 'a340', 'a380', 'aah', 'abonne', 'abord', 'absence', 'abstiens', 'acceder', 'acces', 'accompagnants', 'accompagnee', 'accompagnees', 'accompagnement', 'accompagner', 'accord', 'accordee', 'accueil', 'accueillent', 'accueillir', 'accueillis', 'achat', 'achats', 'acheter', 'acteurs', 'actions', 'active', 'actives', 'activite', 'activites', 'actualise', 'actualisees', 'actuel', 'actuelle', 'actuellement', 'adaptation', 'adapte', 'adaptee', 'adapter', 'adil', 'admis', 'adopte', 'adopter', 'adresse', 'adressent', 'adresser', 'adulte', 'aeeh', 'aerien', 'aerienne', 'aeriennes', 'aeroport', 'aeroports', 'affaires', 

Définition d ela fonction à appeler dans le chatbot :

In [12]:
# on définit la fonction qu'on appellera dans le chatbot : elle renvoie 
# la phrase la plus proche de celle posée par l'utilisateur
from sklearn.metrics.pairwise import cosine_similarity

def reponse_wiki(phrase_user):
    # on a besoin d epasser la chaîne de caractère dans une liste :
    phrase_user = [phrase_user]
    # On calcule les valuers TF-IDF pour la phrase de l'utilisateur
    user_tf = tfidf.transform(phrase_user)
    # on calcule la similarité entre la question posée par l'utilisateur
    # et l'ensemble des phrases de la page wiki
    similarity = cosine_similarity(user_tf, phrases_tf).flatten()
    # on sort l'index de la phrase étant la plus similaire
    index_max_sim = np.argmax(similarity)
    # Si la similarité max ets égale à 0 == pas de correspondance trouvée
    if(similarity[index_max_sim] == 0):
        robo_response = "I didn't find this info, sorry"
    # Sinon, on sort la phrase correspondant le plus : 
    else:
        robo_response = sent_tokens[index_max_sim]
    return robo_response

Définition du chatbot :

In [16]:
# On a plus qu'à utiliser cette fonction dans un chatbot : 
print("""Hi! I can answer your questions about chatbots!
      Type quit to quit""")
flag = True
while (flag == True):
    phrase_user = input("> ")
    phrase_user = phrase_user.lower()
    if (phrase_user == "quit"):
        print ("Bye!")
        flag = False
    else:
        print(reponse_wiki(phrase_user))

Hi! I can answer your questions about chatbots!
      Type quit to quit
> fait-il se laver les main?
Face à l’épidémie , il existe des gestes simples pour préserver votre santé et celle de votre entourage :

Se laver les mains très régulièrement
Tousser ou éternuer dans son coude ou dans un mouchoir 
Saluer sans se serrer la main, arrêter les embrassades
Utiliser des mouchoirs à usage unique et les jeter
Eviter les rassemblements, limiter les déplacements et les contacts.
> quit
Bye!
